In [1]:
import torch as t
import torch.nn as nn

#corr2d函数实现二维互相关运算
def corr2d(X,K):
    H,W=X.shape
    h,w=K.shape
    Y=t.zeros(H-h+1,W-w+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y

In [2]:
X=t.tensor([[0,1,2],[3,4,5],[6,7,8]])
K=t.tensor([[0,1],[2,3]])
Y=corr2d(X,K)
print(Y)

tensor([[19., 25.],
        [37., 43.]])


In [3]:
#二维卷积层
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super(Conv2D,self).__init__()
        self.weight=nn.Parameter(t.randn(kernel_size))
        self.bias=nn.Parameter(t.randn(1))
        
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

In [11]:
X=t.ones(6,8)
Y=t.zeros(6,7)
X[:,2:6]=0
Y[:,1]=1
Y[:,5]=-1
print(X)
print(Y)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


#我们希望学习一个$1 \times 2$卷积层，通过卷积层来检测颜色边缘。

In [12]:
conv2d=Conv2D(kernel_size=(1,2))#因为检测的是水平边缘，故没必要卷积核高超过1，宽为2来检测水平颜色变化
step=30
lr=0.01
for i in range(step):
    Y_hat=conv2d(X)
    l=((Y_hat-Y)**2).sum()#batch_size=1
    l.backward()
    #梯度下降
    conv2d.weight.data-=lr*conv2d.weight.grad
    conv2d.bias.data-=lr*conv2d.bias.grad
    
    #梯度清零
    conv2d.weight.grad.zero_()
    conv2d.bias.grad.zero_()
    if (i+1)%5==0:
        print('Step %d,loss %.3f'%(i+1,l.item()))
    
print(conv2d.weight.data)
print(conv2d.bias.data)

Step 5,loss 10.875
Step 10,loss 2.898
Step 15,loss 0.793
Step 20,loss 0.219
Step 25,loss 0.061
Step 30,loss 0.017
tensor([[ 0.9664, -0.9675]])
tensor([0.0006])


In [13]:
Y_hat=conv2d(X)
print(Y_hat)

tensor([[-4.5833e-04,  9.6702e-01,  5.8284e-04,  5.8284e-04,  5.8284e-04,
         -9.6690e-01, -4.5833e-04],
        [-4.5833e-04,  9.6702e-01,  5.8284e-04,  5.8284e-04,  5.8284e-04,
         -9.6690e-01, -4.5833e-04],
        [-4.5833e-04,  9.6702e-01,  5.8284e-04,  5.8284e-04,  5.8284e-04,
         -9.6690e-01, -4.5833e-04],
        [-4.5833e-04,  9.6702e-01,  5.8284e-04,  5.8284e-04,  5.8284e-04,
         -9.6690e-01, -4.5833e-04],
        [-4.5833e-04,  9.6702e-01,  5.8284e-04,  5.8284e-04,  5.8284e-04,
         -9.6690e-01, -4.5833e-04],
        [-4.5833e-04,  9.6702e-01,  5.8284e-04,  5.8284e-04,  5.8284e-04,
         -9.6690e-01, -4.5833e-04]], grad_fn=<AddBackward0>)


In [16]:
X=t.rand(4,2,3,5)
print(X.shape)

conv2d=nn.Conv2d(in_channels=2,out_channels=3,kernel_size=(3, 5), stride=1, padding=(1, 2))
Y = conv2d(X)
print('Y.shape: ', Y.shape)
print('weight.shape: ', conv2d.weight.shape)
print('bias.shape: ', conv2d.bias.shape)

torch.Size([4, 2, 3, 5])
Y.shape:  torch.Size([4, 3, 3, 5])
weight.shape:  torch.Size([3, 2, 3, 5])
bias.shape:  torch.Size([3])


In [19]:
X = t.arange(32, dtype=t.float32).view(1, 2, 4, 4)
pool2d = nn.MaxPool2d(kernel_size=3, padding=1, stride=(2, 1))
Y = pool2d(X)
print(X)
print(Y.shape)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[16., 17., 18., 19.],
          [20., 21., 22., 23.],
          [24., 25., 26., 27.],
          [28., 29., 30., 31.]]]])
torch.Size([1, 2, 2, 4])
